In [ ]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB,MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_auc_score,accuracy_score
import emoji

In [ ]:
# importing data
df=pd.read_csv('train-v2.tsv',sep='\t',header=None)
df.head()

,0,1
0,0,@USER @USER a sicrhau bod mwy o arian poced 'd...
1,1,Parti Dolig da gyda tim swyddfa canolog @USER ...
2,0,@USER yeaah ma fe yn wir. ( oh well.
3,1,@USER hahaha idk. 3am oedd y bws ti?
4,0,@USER dwim yn gal llun ohoni?


In [ ]:
# importing test data set
dft=pd.read_csv('test.tsv',sep='\t',header=None)
dft.head()

,0,1
0,0,"@USER @USER nos sadwrn dwi fyd, rudimental a e..."
1,1,@USER @USER haia! Pob hwyl fory hogs. Cofiwch ...
2,0,Loner trwy'r dydd heddiw yn ddiolchgar am cwmm...
3,1,"@USER good news, ma'r braid yn cal aros miwn! ..."
4,1,@USER Wi'n gobeithio ti'n cael nadolig gwych g...


In [ ]:
df.describe()
dft.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9672 entries, 0 to 9671
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       9672 non-null   int64 
 1   1       9672 non-null   object
dtypes: int64(1), object(1)
memory usage: 151.2+ KB


In [ ]:
# I tried a list on words that i got from wikipedia (i converted them into a csv file)
dl=pd.read_csv('welshstpwxlsx.csv',header=None)
dl[0]

0         yn
1          y
2          i
3          a
4         'r
       ...  
95      rhyw
96       pen
97    meddai
98       byd
99       yno
Name: 0, Length: 100, dtype: object

In [ ]:
#list of stop words
stopwords = dl[0].tolist()
stopwords.remove('Cymru')
stopwords.remove('Cymraeg')

stopwords

['yn',
 'y',
 'i',
 'a',
 "'r",
 'o',
 'ar',
 'ei',
 "'n",
 'yr',
 'ac',
 'oedd',
 'bod',
 'mae',
 'am',
 'wedi',
 'ond',
 'un',
 "'i",
 'eu',
 'gan',
 'fel',
 'mewn',
 'â',
 'roedd',
 'cael',
 'yw',
 'wrth',
 'ni',
 'hyn',
 'na',
 'ôl',
 'hynny',
 'fe',
 'er',
 'neu',
 'nid',
 'at',
 'sy',
 "'w",
 'hi',
 'dim',
 'mynd',
 'byddai',
 'gyda',
 'yng',
 'iawn',
 'pob',
 'lle',
 'pan',
 'bu',
 'gwaith',
 'mawr',
 'rhai',
 'ym',
 'fy',
 'bydd',
 'gwneud',
 'hyd',
 'hefyd',
 'mi',
 'mai',
 'sydd',
 'gweld',
 'trwy',
 'iddo',
 'arall',
 'yna',
 "'u",
 'mwy',
 'nhw',
 'ein',
 'hwn',
 'hen',
 'mor',
 'yma',
 'cyn',
 'rhaid',
 'nad',
 'pobl',
 'ef',
 'dau',
 'rhan',
 'llawer',
 'chi',
 'os',
 'rhoi',
 'daeth',
 'hun',
 'cyntaf',
 'newydd',
 'felly',
 'tros',
 'rhyw',
 'pen',
 'meddai',
 'byd',
 'yno']

In [ ]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78609 entries, 0 to 78608
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       78609 non-null  int64 
 1   1       78609 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.2+ MB


,0
count,78609.000000
mean,0.500312
std,0.500003
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [ ]:
df.columns = ['index','welsh tweets']
df.head()
dft.columns = ['index','welsh tweets']
dft.head()

,index,welsh tweets
0,0,"@USER @USER nos sadwrn dwi fyd, rudimental a e..."
1,1,@USER @USER haia! Pob hwyl fory hogs. Cofiwch ...
2,0,Loner trwy'r dydd heddiw yn ddiolchgar am cwmm...
3,1,"@USER good news, ma'r braid yn cal aros miwn! ..."
4,1,@USER Wi'n gobeithio ti'n cael nadolig gwych g...


In [ ]:
#converting all the strings into lower case letters.
df_lower = df["welsh tweets"].map(lambda x: x.lower())
#converting all the strings into lower case letters.
dft_lower = dft["welsh tweets"].map(lambda x: x.lower())

In [ ]:
df_lower_tweets = pd.DataFrame(df_lower)
dft_lower_tweets = pd.DataFrame(dft_lower)

In [ ]:
# this is the code for replacing emojis with text
df_emoji = df_lower_tweets['welsh tweets'].apply(lambda x: emoji.demojize(x))
# this is the code for replacing emojis with text
dft_emoji = dft_lower_tweets['welsh tweets'].apply(lambda x: emoji.demojize(x))


In [ ]:
df_no_emoji_lower = pd.DataFrame(df_emoji)
dft_no_emoji_lower = pd.DataFrame(dft_emoji)

In [ ]:
df_no_emoji_lower

,welsh tweets
0,@user @user a sicrhau bod mwy o arian poced 'd...
1,parti dolig da gyda tim swyddfa canolog @user ...
2,@user yeaah ma fe yn wir. ( oh well.
3,@user hahaha idk. 3am oedd y bws ti?
4,@user dwim yn gal llun ohoni?
...,...
78604,"@user @user diolch yn fawr am ein diweddaru, r..."
78605,@user @user @user diolch! digon o opsiynne felly!
78606,mor hungover heddiw :water_pistol:
78607,"@user eidiaaaal, awni dydd llun nesa ia? y air..."


In [ ]:
df_clean = pd.concat([df, df_no_emoji_lower], axis=1, ignore_index=True)
df_clean
dft_clean = pd.concat([dft, dft_no_emoji_lower], axis=1, ignore_index=True)
dft_clean

,0,1,2
0,0,"@USER @USER nos sadwrn dwi fyd, rudimental a e...","@user @user nos sadwrn dwi fyd, rudimental a e..."
1,1,@USER @USER haia! Pob hwyl fory hogs. Cofiwch ...,@user @user haia! pob hwyl fory hogs. cofiwch ...
2,0,Loner trwy'r dydd heddiw yn ddiolchgar am cwmm...,loner trwy'r dydd heddiw yn ddiolchgar am cwmm...
3,1,"@USER good news, ma'r braid yn cal aros miwn! ...","@user good news, ma'r braid yn cal aros miwn! ..."
4,1,@USER Wi'n gobeithio ti'n cael nadolig gwych g...,@user wi'n gobeithio ti'n cael nadolig gwych g...
...,...,...,...
9667,1,@USER Ahh scary D: Swni'n ofn mynd mwy na 10mp...,@user ahh scary d: swni'n ofn mynd mwy na 10mp...
9668,0,Ma'n neud i fi deimlo bach yn sâl mod i rwan w...,ma'n neud i fi deimlo bach yn sâl mod i rwan w...
9669,0,"So ma Bac fi di gal ei ddewis i yrru ffwrdd, gret","so ma bac fi di gal ei ddewis i yrru ffwrdd, gret"
9670,1,Bore da a diolch am ein dilyn @USER a @USER! B...,bore da a diolch am ein dilyn @user a @user! b...


In [ ]:
df_clean.columns = ['index','unclean tweets','welsh tweets']
dft_clean.columns = ['index','unclean tweets','welsh tweets']

In [ ]:
df_final = df_clean.drop('unclean tweets',axis=1)
dft_final = dft_clean.drop('unclean tweets',axis=1)

In [ ]:
data = df_final
datat = dft_final

In [ ]:
# i tried removing special characters(punctuation marks which are listed below) from dataframe
data['welsh tweets'] = data['welsh tweets'].str.replace(r"[!$%&'()*+,-./:;<=>?[\]^_`{|}~]", '',regex=True)
data
# i tried removing special characters from dataframe
datat['welsh tweets'] = datat['welsh tweets'].str.replace(r"[!$%&'()*+,-./:;<=>?[\]^_`{|}~]", '',regex=True)
datat

,index,welsh tweets
0,0,@user @user nos sadwrn dwi fyd rudimental a el...
1,1,@user @user haia pob hwyl fory hogs cofiwch we...
2,0,loner trwyr dydd heddiw yn ddiolchgar am cwmmn...
3,1,@user good news mar braid yn cal aros miwn hah...
4,1,@user win gobeithio tin cael nadolig gwych gyd...
...,...,...
9667,1,@user ahh scary d swnin ofn mynd mwy na 10mph ...
9668,0,man neud i fi deimlo bach yn sâl mod i rwan we...
9669,0,so ma bac fi di gal ei ddewis i yrru ffwrdd gret
9670,1,bore da a diolch am ein dilyn @user a @user bw...


In [ ]:
# # tried removing @USER
data['welsh tweets'] = data['welsh tweets'].str.replace(r'@user', '')

In [ ]:
# # tried removing @USER
datat['welsh tweets'] = datat['welsh tweets'].str.replace(r'@user', '')

In [ ]:
# # tried removing URL
data['welsh tweets'] = data['welsh tweets'].str.replace(r'url', '')

In [ ]:
# # tried removing URL
datat['welsh tweets'] = datat['welsh tweets'].str.replace(r'url', '')

In [ ]:
# tried dropping duplicate elements
# df_final = df_final.drop_duplicates()
# df_final

In [ ]:
data_final = data.drop_duplicates()
data_final

,index,welsh tweets
0,0,a sicrhau bod mwy o arian poced da ti nes i ...
1,1,parti dolig da gyda tim swyddfa canolog ty gw...
2,0,yeaah ma fe yn wir oh well
3,1,hahaha idk 3am oedd y bws ti
4,0,dwim yn gal llun ohoni
...,...,...
78604,1,diolch yn fawr am ein diweddaru rôn in amau ...
78605,1,diolch digon o opsiynne felly
78606,0,mor hungover heddiw waterpistol
78607,1,eidiaaaal awni dydd llun nesa ia y airoplane ...


In [ ]:
datat_final = datat.drop_duplicates()
datat_final

,index,welsh tweets
0,0,nos sadwrn dwi fyd rudimental a ella yn supp...
1,1,haia pob hwyl fory hogs cofiwch wenu welai c...
2,0,loner trwyr dydd heddiw yn ddiolchgar am cwmmn...
3,1,good news mar braid yn cal aros miwn haha dwi...
4,1,win gobeithio tin cael nadolig gwych gyda eic...
...,...,...
9667,1,ahh scary d swnin ofn mynd mwy na 10mph aye d...
9668,0,man neud i fi deimlo bach yn sâl mod i rwan we...
9669,0,so ma bac fi di gal ei ddewis i yrru ffwrdd gret
9670,1,bore da a diolch am ein dilyn a bwrw golwg a...


In [ ]:
# train set
vectorizer = CountVectorizer(min_df=2,stop_words=stopwords,max_features=2000,ngram_range=(1,2))
features = vectorizer.fit_transform(data_final['welsh tweets'])
indexs = data_final['index']

In [ ]:
#test set
vectorizer = CountVectorizer(min_df=2,stop_words=stopwords,max_features=2000,ngram_range=(1,2))
x_test_final = vectorizer.fit_transform(datat_final['welsh tweets'])
y_test_final = datat_final['index']

In [ ]:
# vectorizer = TfidfVectorizer(max_features=2000)
# features = vectorizer.fit_transform(df['welsh tweets'])
# features

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features,indexs, test_size=0.1,random_state=666)

In [ ]:
naivebayes_bernoulli = BernoulliNB()
naivebayes_bernoulli.fit(x_train.toarray(), y_train)

BernoulliNB()

In [ ]:
naivebayes_multinomial = MultinomialNB()
naivebayes_multinomial.fit(x_train.toarray(), y_train)

MultinomialNB()

In [ ]:
lr = LogisticRegression(max_iter=1000)
lr.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
nbbpredict = naivebayes_bernoulli.predict(x_test.toarray())

In [ ]:
nbmpredict = naivebayes_multinomial.predict(x_test.toarray())

In [ ]:
lrpredict = lr.predict(x_test)

In [ ]:
x_prob = lr.predict_proba(x_test)

In [ ]:
##### validation accuracy ########

In [ ]:
print(metrics.classification_report(y_test,nbmpredict))

              precision    recall  f1-score   support

           0       0.72      0.73      0.72      3953
           1       0.71      0.70      0.71      3836

    accuracy                           0.72      7789
   macro avg       0.72      0.72      0.72      7789
weighted avg       0.72      0.72      0.72      7789



In [ ]:
print(metrics.classification_report(y_test,nbbpredict))

              precision    recall  f1-score   support

           0       0.72      0.70      0.71      3953
           1       0.70      0.73      0.71      3836

    accuracy                           0.71      7789
   macro avg       0.71      0.71      0.71      7789
weighted avg       0.71      0.71      0.71      7789



In [ ]:
from sklearn.metrics import f1_score,recall_score
print(metrics.classification_report(y_test,lrpredict))
print(roc_auc_score(y_test, x_prob[:, 1]))
print(accuracy_score(y_test,lrpredict))
print(recall_score(y_test,lrpredict))


              precision    recall  f1-score   support

           0       0.74      0.72      0.73      3953
           1       0.72      0.74      0.73      3836

    accuracy                           0.73      7789
   macro avg       0.73      0.73      0.73      7789
weighted avg       0.73      0.73      0.73      7789

0.8021720347028576
0.7255103350879445
0.7356621480709072


In [ ]:
########## test set accuracy ###################

In [ ]:
nbbpredict_final = naivebayes_bernoulli.predict(x_test_final.toarray())

In [ ]:
print(metrics.classification_report(y_test_final,nbbpredict_final))

              precision    recall  f1-score   support

           0       0.59      0.62      0.60      4917
           1       0.58      0.55      0.57      4725

    accuracy                           0.59      9642
   macro avg       0.59      0.59      0.59      9642
weighted avg       0.59      0.59      0.59      9642



In [ ]:
nbmpredict_final = naivebayes_multinomial.predict(x_test_final.toarray())

In [ ]:
print(metrics.classification_report(y_test_final,nbmpredict_final))

              precision    recall  f1-score   support

           0       0.58      0.65      0.62      4917
           1       0.59      0.51      0.55      4725

    accuracy                           0.58      9642
   macro avg       0.58      0.58      0.58      9642
weighted avg       0.58      0.58      0.58      9642



In [ ]:
lrpredict_final = lr.predict(x_test_final)

In [ ]:
x_test_final

<9642x2000 sparse matrix of type '<class 'numpy.int64'>'
	with 69891 stored elements in Compressed Sparse Row format>

In [ ]:
y_test_final

0       0
1       1
2       0
3       1
4       1
       ..
9667    1
9668    0
9669    0
9670    1
9671    0
Name: index, Length: 9642, dtype: int64

In [ ]:
x_prob_final = lr.predict_proba(x_test_final)

In [ ]:
from sklearn.metrics import f1_score,recall_score
print(metrics.classification_report(y_test_final,lrpredict_final))
print(roc_auc_score(y_test_final, x_prob_final[:, 1]))
print(accuracy_score(y_test_final,lrpredict_final))
print(recall_score(y_test_final,lrpredict_final))


              precision    recall  f1-score   support

           0       0.57      0.60      0.59      4917
           1       0.56      0.52      0.54      4725

    accuracy                           0.56      9642
   macro avg       0.56      0.56      0.56      9642
weighted avg       0.56      0.56      0.56      9642

0.5864049249284149
0.5644057249533292
0.5225396825396825


In [ ]:
lr = LogisticRegression(max_iter=2000)
lr.fit(features, indexs)

LogisticRegression(max_iter=2000)

In [ ]:
predlr = lr.predict(x_test_final)

In [ ]:
accuracy_score(y_test_final,predlr)

0.566894835096453

In [ ]:
#sources:
#https://betterprogramming.pub/using-emoji-to-convert-emojis-to-text-f1969352083c
#https://www.analyticsvidhya.com/blog/2021/09/sentiment-classification-using-nlp-with-text-analytics/

AT the end i noticed overfitting on certain kinds of data, (Probably due to too much prepeocessing), is this kind of preprocessing redundant or does this have any methods that can make the accuracy for all values better.
